In [11]:
from libraries import *
import spacy
from tqdm import tqdm  

In [12]:
doc = fitz.open("book.pdf")

In [13]:
chapter_starts_d = {"Chapter 1": 26,
                  "Chapter 2": 70,
                  "Chapter 3": 105,
                  "Chapter 4": 145,
                  "Chapter 5": 190,
                  "Chapter 6": 219,
                  "Chapter 7": 253,
                  "Chapter 8": 300,
                  "Chapter 9": 343,
                  "Chapter 10": 353,
                  "Chapter 11": 397,
                  "Chapter 12": 437,
                  "Chapter 13": 481,
                  "Chapter 14": 513,
                  "Chapter 15": 560,
                  "Chapter 16": 600,
                  "Chapter 17": 638,
                  "Chapter 18": 681,
                  "Chapter 19": 713,
                  "Chapter 20": 732,
                  "Chapter 21": 748,
                  "Out of chapter 21" : 793}
chapter_starts = list(chapter_starts_d.values())
def extract_chapters_from_pdf(pdf_path, chapter_starts):
    doc = fitz.open(pdf_path)
    chapter_dict = {}
    num_chapters = len(chapter_starts) - 1  # On suppose que la dernière entrée est la fin du dernier chapitre

    # Extraire le texte pour chaque chapitre
    for i in range(num_chapters):
        start_page = chapter_starts[i]
        end_page = chapter_starts[i + 1] - 1  # La page de fin est la page de début du prochain chapitre moins 1
        text = ""

        # Concaténer le texte de chaque page du chapitre
        for page_num in range(start_page, end_page + 1):  # +1 pour inclure la page de fin
            page = doc.load_page(page_num)
            text += page.get_text()

        chapter_dict[i + 1] = text  # Utiliser un nom générique avec numéro de chapitre

    return chapter_dict
    

In [14]:
chapter_dict = extract_chapters_from_pdf(pdf_path="book.pdf", chapter_starts=chapter_starts)

In [15]:
chapter_dict[1]

'2\u2002 \u2002 Chapter 1 / Basic Concepts and Computer Evolution \n\t 1.1\t Organization and Architecture\nIn describing computers, a distinction is often made between computer architec-\nture and computer organization. Although it is difficult to give precise definitions \nfor these terms, a consensus exists about the general areas covered by each. For \nexample, see [VRAN80], [SIEW82], and [BELL78a]; an interesting alternative view \nis presented in [REDD76].\nComputer architecture refers to those attributes of a system visible to a pro-\ngrammer or, put another way, those attributes that have a direct impact on the \nlogical execution of a program. A term that is often used interchangeably with com-\nputer architecture is instruction set architecture (ISA). The ISA defines instruction \nformats, instruction opcodes, registers, instruction and data memory; the effect of \nexecuted instructions on the registers and memory; and an algorithm for control-\nling instruction execution. Co

In [16]:
len(chapter_dict[1])

109820

In [17]:
# load spacy
nlp = spacy.load('en_core_web_sm')

In [18]:
doc = nlp(chapter_dict[1])

In [19]:
sentences = [sent.text for sent in doc.sents if len(sent.text.strip()) > 0]

In [20]:
summaries = []
for sentence in tqdm(sentences, desc="Summarizing sentences"):
    summaries.append(summarize(sentence))

Summarizing sentences: 100%|██████████| 789/789 [36:38<00:00,  2.79s/it]


In [21]:
final_summary = summarize(" ".join(summaries))
print(final_summary)

In describing computers, a distinction is often made between computer architec-ture and computer organization. Computer organization refers to the operational units and their interconnections that realize the architectural specifications. Computer architecture refers to those attributes of a system visible to a pro-grammer.


In [24]:
summary_concat = " ".join(summaries)
summary_concat

'2\u2002 \u2002 Chapter 1 / Basic Concepts and Computer Evolution \t 1.1\t Organization and Architecture of the Computer World. 1.2\u2009 \u2002 \u2009 \u2009 Basic Concepts of Computer Evolution. \u2009 Organization and architecture of the computer world. In describing computers, a distinction is often made between computer architec-ture and computer organization. A distinction is made between "architec" and "organization" in terms of the way computers are used. The term is often used to refer to computers that are used for business purposes. Although it is difficult to give precise definitions for these terms, a consensus exists about the general areas covered by each. For example, the term "sociology" is used to refer to a range of fields of study. For more information on the terms, see:\xa0"Sociology.com" For  example, see [VRAN80], [SIEW82], and [BELL78a]; an interesting alternative view is presented in [REDD76]. For example, \xa0for example, [VRan80] \xa0is presented as an altern

In [27]:
doc

2    Chapter 1 / Basic Concepts and Computer Evolution 
	 1.1	 Organization and Architecture
In describing computers, a distinction is often made between computer architec-
ture and computer organization. Although it is difficult to give precise definitions 
for these terms, a consensus exists about the general areas covered by each. For 
example, see [VRAN80], [SIEW82], and [BELL78a]; an interesting alternative view 
is presented in [REDD76].
Computer architecture refers to those attributes of a system visible to a pro-
grammer or, put another way, those attributes that have a direct impact on the 
logical execution of a program. A term that is often used interchangeably with com-
puter architecture is instruction set architecture (ISA). The ISA defines instruction 
formats, instruction opcodes, registers, instruction and data memory; the effect of 
executed instructions on the registers and memory; and an algorithm for control-
ling instruction execution. Computer organization refers